# PREGUNTA 3

In [12]:
# Definición de los parámetros y casos
casos = [
    ((0, 16), 0),
    ((3, 8), -43),
    ((1/2, 1/2), 1/4)
]

# Proceso para cada caso
for P, a in casos:
    x, y = P
    orden = None
    for i in range(1, 100 + 1):
        # Inicializar R y Q
        R = "i"
        Q = P
        k = i
        while k > 0:
            if k % 2 == 1:
                if R == "i":
                    R = Q
                else:
                    if Q == "i":
                        pass
                    else:
                        x1, y1 = R
                        x2, y2 = Q
                        if x1 == x2 and y1 == -y2:
                            R = "i"
                        else:
                            if R == Q:
                                if y1 == 0:  # Evitar división por cero
                                    R = "i"
                                else:
                                    m = (3 * x1 ** 2 + a) / (2 * y1)
                            else:
                                if x2 - x1 == 0:  # Evitar división por cero
                                    R = "i"
                                else:
                                    m = (y2 - y1) / (x2 - x1)
                            if R != "i":
                                x3 = (m ** 2 - x1 - x2)
                                y3 = (m * (x1 - x3) - y1)
                                R = (x3, y3)
            if Q != "i":
                x1, y1 = Q
                if y1 == 0:  # Evitar división por cero
                    Q = "i"
                else:
                    m = (3 * x1 ** 2 + a) / (2 * y1)
                    x2, y2 = Q
                    x3 = (m ** 2 - x1 - x2)
                    y3 = (m * (x1 - x3) - y1)
                    Q = (x3, y3)
            k //= 2
        if R == "i":
            orden = i
            break
    
    # Impresión del resultado
    print(f"El orden de P{P} es: {orden}")


El orden de P(0, 16) es: 3
El orden de P(3, 8) es: 7
El orden de P(0.5, 0.5) es: 4


# PREGUNTA 4

In [16]:
def mod_inv(a, p):
    """Compute the modular inverse of a modulo p"""
    return pow(a, p - 2, p)

class EllipticCurve:
    def __init__(self, a, b, p):
        self.a = a
        self.b = b
        self.p = p

    def is_on_curve(self, x, y):
        return (y**2 - (x**3 + self.a * x + self.b)) % self.p == 0

    def point_addition(self, P, Q):
        if P is None:
            return Q
        if Q is None:
            return P
        x1, y1 = P
        x2, y2 = Q
        if x1 == x2 and y1 != y2:
            return None  # Point at infinity
        if x1 == x2:
            m = (3 * x1**2 + self.a) * mod_inv(2 * y1, self.p) % self.p
        else:
            m = (y2 - y1) * mod_inv(x2 - x1, self.p) % self.p
        x3 = (m**2 - x1 - x2) % self.p
        y3 = (m * (x1 - x3) - y1) % self.p
        return (x3, y3)

    def point_doubling(self, P):
        if P is None:
            return None
        x1, y1 = P
        if y1 == 0:
            return None  # Point at infinity
        m = (3 * x1**2 + self.a) * mod_inv(2 * y1, self.p) % self.p
        x3 = (m**2 - 2 * x1) % self.p
        y3 = (m * (x1 - x3) - y1) % self.p
        return (x3, y3)

    def scalar_multiplication(self, k, P):
        result = None
        addend = P

        while k:
            if k & 1:
                result = self.point_addition(result, addend)
            addend = self.point_doubling(addend)
            k >>= 1

        return result

    def find_points(self):
        points = []
        for x in range(self.p):
            for y in range(self.p):
                if self.is_on_curve(x, y):
                    points.append((x, y))
        return points

# Definir la curva elíptica E(Z_13) : y^2 = x^3 + 10x + 6
curve = EllipticCurve(10, 6, 13)

# Parte (a): Encontrar todos los elementos de E(Z_13)
points = curve.find_points()
print("Puntos en E(Z_13):")
print(points)

# Parte (b): Cifrar el mensaje Q = (11, 11) usando ECES
d = 4
k = 7
P = (5, 5)
Q = (11, 11)

dP = curve.scalar_multiplication(d, P)
kP = curve.scalar_multiplication(k, P)
k_dP = curve.scalar_multiplication(k, dP)
c1 = kP
c2 = curve.point_addition(Q, k_dP)

print(f"c1 = {c1}")
print(f"c2 = {c2}")


Puntos en E(Z_13):
[(1, 2), (1, 11), (5, 5), (5, 8), (6, 3), (6, 10), (7, 4), (7, 9), (8, 0), (10, 1), (10, 12), (11, 2), (11, 11)]
c1 = None
c2 = (11, 11)


# PREGUNTA 5

In [17]:
# Parámetros de la curva y valores iniciales
a = 2
b = 2
p = 17  # Campo 
G = (5, 1)  # Punto de referencia
n = 19  # Orden previsto
d = 10  # Clave priv

# Inverso modular usando pow
mod_inv = lambda a, p: pow(a, p - 2, p)

# Generación de la clave pub
Q = None
k = d
Q_temp = G
while k > 0:
    if k % 2 == 1:
        if Q is None:
            Q = Q_temp
        else:
            (x1, y1), (x2, y2) = Q, Q_temp
            if Q == Q_temp:
                m = (3 * x1 ** 2 + a) * mod_inv(2 * y1, p) % p
            else:
                m = (y2 - y1) * mod_inv(x2 - x1, p) % p
            x3 = (m ** 2 - x1 - x2) % p
            y3 = (m * (x1 - x3) - y1) % p
            Q = (x3, y3)
    (x1, y1), (x2, y2) = Q_temp, Q_temp
    m = (3 * x1 ** 2 + a) * mod_inv(2 * y1, p) % p
    x3 = (m ** 2 - x1 - x2) % p
    y3 = (m * (x1 - x3) - y1) % p
    Q_temp = (x3, y3)
    k //= 2

# Lista de casos
casos = [
    {"h_m": 12, "k": 11},
    {"h_m": 4, "k": 13},
    {"h_m": 9, "k": 8}
]

# Proceso para cada caso
for caso in casos:
    e = caso["h_m"]
    k = caso["k"]
    
    # Firmar el mensaje utilizando ECDSA
    R = None
    R_temp = G
    k_temp = k
    while k_temp > 0:
        if k_temp % 2 == 1:
            if R is None:
                R = R_temp
            else:
                (x1, y1), (x2, y2) = R, R_temp
                if R == R_temp:
                    m = (3 * x1 ** 2 + a) * mod_inv(2 * y1, p) % p
                else:
                    m = (y2 - y1) * mod_inv(x2 - x1, p) % p
                x3 = (m ** 2 - x1 - x2) % p
                y3 = (m * (x1 - x3) - y1) % p
                R = (x3, y3)
        (x1, y1), (x2, y2) = R_temp, R_temp
        m = (3 * x1 ** 2 + a) * mod_inv(2 * y1, p) % p
        x3 = (m ** 2 - x1 - x2) % p
        y3 = (m * (x1 - x3) - y1) % p
        R_temp = (x3, y3)
        k_temp //= 2
    
    r = R[0] % n
    s = (mod_inv(k, n) * (e + r * d)) % n

    # Verificar la firma utilizando ECDSA
    w = mod_inv(s, n)
    u1 = (e * w) % n
    u2 = (r * w) % n

    P1 = None
    P1_temp = G
    u1_temp = u1
    while u1_temp > 0:
        if u1_temp % 2 == 1:
            if P1 is None:
                P1 = P1_temp
            else:
                (x1, y1), (x2, y2) = P1, P1_temp
                if P1 == P1_temp:
                    m = (3 * x1 ** 2 + a) * mod_inv(2 * y1, p) % p
                else:
                    m = (y2 - y1) * mod_inv(x2 - x1, p) % p
                x3 = (m ** 2 - x1 - x2) % p
                y3 = (m * (x1 - x3) - y1) % p
                P1 = (x3, y3)
        (x1, y1), (x2, y2) = P1_temp, P1_temp
        m = (3 * x1 ** 2 + a) * mod_inv(2 * y1, p) % p
        x3 = (m ** 2 - x1 - x2) % p
        y3 = (m * (x1 - x3) - y1) % p
        P1_temp = (x3, y3)
        u1_temp //= 2

    P2 = None
    P2_temp = Q
    u2_temp = u2
    while u2_temp > 0:
        if u2_temp % 2 == 1:
            if P2 is None:
                P2 = P2_temp
            else:
                (x1, y1), (x2, y2) = P2, P2_temp
                if P2 == P2_temp:
                    m = (3 * x1 ** 2 + a) * mod_inv(2 * y1, p) % p
                else:
                    m = (y2 - y1) * mod_inv(x2 - x1, p) % p
                x3 = (m ** 2 - x1 - x2) % p
                y3 = (m * (x1 - x3) - y1) % p
                P2 = (x3, y3)
        (x1, y1), (x2, y2) = P2_temp, P2_temp
        m = (3 * x1 ** 2 + a) * mod_inv(2 * y1, p) % p
        x3 = (m ** 2 - x1 - x2) % p
        y3 = (m * (x1 - x3) - y1) % p
        P2_temp = (x3, y3)
        u2_temp //= 2

    if P1 is None:
        P1 = P2
    else:
        (x1, y1), (x2, y2) = P1, P2
        if P1 == P2:
            m = (3 * x1 ** 2 + a) * mod_inv(2 * y1, p) % p
        else:
            m = (y2 - y1) * mod_inv(x2 - x1, p) % p
        x3 = (m ** 2 - x1 - x2) % p
        y3 = (m * (x1 - x3) - y1) % p
        P1 = (x3, y3)

    valido = P1 is not None and P1[0] % n == r
    
    print(f"Para h(m) = {e}, k = {k}:")
    print("  Firma: (r, s) =", (r, s))
    print("  ¿Firma válida?", valido)
    print()


Para h(m) = 12, k = 11:
  Firma: (r, s) = (13, 6)
  ¿Firma válida? True

Para h(m) = 4, k = 13:
  Firma: (r, s) = (16, 17)
  ¿Firma válida? True

Para h(m) = 9, k = 8:
  Firma: (r, s) = (13, 15)
  ¿Firma válida? True

